# 🦜🔗​ Descubriendo el Poder de ​Langchain: 
<img style="float: left;" src="https://bennewmark.wordpress.com/wp-content/uploads/2020/09/sorting-hat.jpg" width="200px"/>

En este cuadernillo utilizaremos Langchain para hacer un aplicación inteligente que actue como el sombrero seleccionador de los libros de Harry Potter. El objetivo del sombrero será analizar la personalidad una persona y enviarle a una de las 4 casas de Hogwarts: 
* **Gryffindor**: Valientes, atrevidos, caballerosos y audaces. Valoran la valentía y suelen ser considerados líderes dispuestos a defender lo que es correcto.
* **Hufflepuff**: Leales, trabajadores, justos y pacientes. Los Hufflepuff valoran la dedicación y la amabilidad, con un fuerte sentido de la justicia y la inclusión.
* **Ravenclaw**: Inteligentes, curiosos, creativos y sabios. Los Ravenclaw valoran el aprendizaje, el ingenio y la sabiduría, y siempre buscan el conocimiento y la comprensión.
* **Slytherin**: Ambiciosos, astutos, ingeniosos y decididos. Los Slytherin son impulsivos, suelen fijarse metas altas y valoran la lealtad dentro de su grupo.

# 🐛 Debug ON/OFF

## ☀️ ON

In [ ]:
import langchain

langchain.debug = True

## 🌑 OFF

In [ ]:
import langchain

langchain.debug = False

# 📜 Sources

In [ ]:
from IPython.display import display, Markdown
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.output_parsers.enum import EnumOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from operator import itemgetter

In [ ]:
import re
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate
import warnings
from enum import Enum


# import warning menssages that start with "UserWarning:"
warnings.filterwarnings("always", category=UserWarning, module='transformers')

system_prompt = """
You are an intelligent and helpful assistant. Your goal is to assist the user by providing clear, accurate, and helpful answers to their questions or requests. 
Be polite, concise, and detailed, aiming to offer the best possible solution. When appropriate, explain your reasoning or suggest additional information. 
If you do not have the necessary data, respond honestly and offer to assist in other ways.
"""

def llama_prompt(human, system=system_prompt, **kwargs):
    prompt = ChatPromptTemplate([
            ("system", system),
            ("human", human)
        ],
        partial_variables=kwargs
    )
    return prompt

class HogwartsHouses(Enum):
    GRYFFINDOR  = "gryffindor"
    HUFFLEPUFF = "hufflepuff"
    RAVENCLAW = "ravenclaw"
    SLYTHERIN = "slytherin"

# 🦙 Cargar el Modelo: LLama 3.1

In [ ]:
# Descarga el modelo llama 3.1 (solo es necesario hacerlo una vez)
!ollama pull llama3.1

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.1",
    temperature = 0,
    num_predict = 256
)

# 🔗​ Chains

## 💬 Prompt simple: 
Resumen de un capitulo de un libro

In [ ]:
from langchain.prompts import ChatPromptTemplate

# create prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
            """
            You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
            If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
            """
        ),
        ("user", 
            """
            Act as the sorting hat and tell to what Hogwarts house will {name} go, be brief. Hogwarts houses have the next traits:
            Gryffindor: Brave, daring, chivalrous, and bold. They value courage and are often seen as leaders willing to stand up for what’s right.
            Hufflepuff: Loyal, hardworking, fair, and patient. Hufflepuffs prize dedication and kindness, with a strong sense of justice and inclusivity.
            Ravenclaw: Intelligent, curious, creative, and wise. Ravenclaws value learning, wit, and wisdom, always seeking knowledge and understanding.
            Slytherin: Ambitious, cunning, resourceful, and determined. Slytherins are driven, often setting high goals and valuing loyalty within their group.
            """,     
        ),
    ]
)

chain = prompt | llm

### invoke(): 
Ejecuta una cadena

In [ ]:
chain.invoke({"name": "Will Smith"})

### batch(): 
Ejecuta la misma cadena varias veces con diferentes entradas

In [ ]:
response = chain.batch([
    {"name": "Madonna"},
    {"name": "Marilyn Monroe"},
])

response

### stream():
Delvuelve la salida poco a poco, no es necesario que termine de generar todo el prompt (simula que escribe la ia)

In [ ]:
for t in chain.stream({"name": "Henry VIII"}):
    print(t)

## 💬🦜 Prompt con parser:

### Text

In [ ]:
from langchain.schema.output_parser import StrOutputParser

parser = StrOutputParser()
prompt = llama_prompt(
        """
        Act as the sorting hat and tell to what Hogwarts house will {name} go, be brief. Hogwarts houses have the next traits:
        Gryffindor: Brave, daring, chivalrous, and bold. They value courage and are often seen as leaders willing to stand up for what’s right.
        Hufflepuff: Loyal, hardworking, fair, and patient. Hufflepuffs prize dedication and kindness, with a strong sense of justice and inclusivity.
        Ravenclaw: Intelligent, curious, creative, and wise. Ravenclaws value learning, wit, and wisdom, always seeking knowledge and understanding.
        Slytherin: Ambitious, cunning, resourceful, and determined. Slytherins are driven, often setting high goals and valuing loyalty within their group.
        """,
        system="""
        You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
        If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
        """
    )

chain = prompt | llm | parser

Markdown(chain.invoke({"name": "Alan Turing"}))

### JSON

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama
from typing import List

llm_json = ChatOllama(
    model = "llama3.2",
    temperature = 0.2,
    num_predict = 512,
    format="json"
)

class HousesData(BaseModel):
    
    class PersonAsignment(BaseModel):
        person: str
        house: str
    
    asignments: List[PersonAsignment]

In [ ]:
parser = JsonOutputParser(pydantic_object=HousesData)

prompt = llama_prompt(
    """
    Act as the sorting hat and tell to what Hogwarts house will these famous persons go: {persons}. Hogwarts houses have the next traits:
    Gryffindor: Brave, daring, chivalrous, and bold. They value courage and are often seen as leaders willing to stand up for what’s right.
    Hufflepuff: Loyal, hardworking, fair, and patient. Hufflepuffs prize dedication and kindness, with a strong sense of justice and inclusivity.
    Ravenclaw: Intelligent, curious, creative, and wise. Ravenclaws value learning, wit, and wisdom, always seeking knowledge and understanding.
    Slytherin: Ambitious, cunning, resourceful, and determined. Slytherins are driven, often setting high goals and valuing loyalty within their group.
    Always follow this instructions when answering: 
    {format_instructions}
    """,
    system="""
    You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
    If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
    """,
    format_instructions=parser.get_format_instructions()
)

chain = (prompt | llm_json | parser)

response = chain.invoke({
    "persons": ["Robert Pattinson", "Emma Watson", "Tom Felton", "Katie Leung"]
})
response

#### Alternative chain

In [ ]:
from langchain_core.exceptions import OutputParserException

parser = JsonOutputParser(pydantic_object=HousesData)

alternative_chain = (
    llama_prompt(
        """
        Use your knowledge of the Harry Potter universe and act as the sorting hat and tell to what Hogwarts house will this famous persons go: {persons}.
        {format_instructions}
        """,
        system="""
        Only respond JSON
        """,
        format_instructions=parser.get_format_instructions()
    )
    | llm_json
    | parser
)

chain = (
    llama_prompt("Use your knowledge of the Harry Potter universe and act as the sorting hat and tell to what Hogwarts house will this famous persons go: {persons}.")
    | llm
    | parser
).with_fallbacks(fallbacks=[fix_chain], exceptions_to_handle=(OutputParserException,))

response = chain.invoke({
    "persons": ["Robert Pattinson", "Emma Watson", "Tom Felton", "Katie Leung"]
})
response

### Dates

In [ ]:
from dateutil import parser
from langchain.output_parsers import DatetimeOutputParser

parser = DatetimeOutputParser()

prompt = llama_prompt(
    "Use your knowledge of the Harry Potter universe to user the users question:\n {question} \n Always follow this instruction to respond:\n {format_instructions}",
    system="""You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
    If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
    """,
    format_instructions=parser.get_format_instructions()
)

chain = prompt | llm | parser

response = chain.invoke({
    "question": "When was Harry Potter born ?",
})
response

### Enums

In [ ]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.exceptions import OutputParserException

class HogwartsHouses(Enum):
    GRYFFINDOR  = "gryffindor"
    HUFFLEPUFF = "hufflepuff"
    RAVENCLAW = "ravenclaw"
    SLYTHERIN = "slytherin"

parser = EnumOutputParser(enum=HogwartsHouses)

prompt = llama_prompt(
    """
    Act as the sorting hat and tell to what Hogwarts house will {name} go. Hogwarts houses have the next traits:
    Gryffindor: Brave, daring, chivalrous, and bold. They value courage and are often seen as leaders willing to stand up for what’s right.
    Hufflepuff: Loyal, hardworking, fair, and patient. Hufflepuffs prize dedication and kindness, with a strong sense of justice and inclusivity.
    Ravenclaw: Intelligent, curious, creative, and wise. Ravenclaws value learning, wit, and wisdom, always seeking knowledge and understanding.
    Slytherin: Ambitious, cunning, resourceful, and determined. Slytherins are driven, often setting high goals and valuing loyalty within their group.
    Always follow this instructions when answering:
    {format_instructions} only return one option with no explanation, always reply in lower case.
    """,
    system="""You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
    If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
    """,
    format_instructions=parser.get_format_instructions()
)

chain = (prompt | llm | parser).with_retry(retry_if_exception_type=(OutputParserException,), stop_after_attempt=3)

response = chain.invoke({
    "name": "Albert Einstein"
})
response

### Output-fixing

In [ ]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser
from langchain.output_parsers.fix import OutputFixingParser
from langchain_core.exceptions import OutputParserException

class HogwartsHouses(Enum):
    GRYFFINDOR  = "gryffindor"
    HUFFLEPUFF = "hufflepuff"
    RAVENCLAW = "ravenclaw"
    SLYTHERIN = "slytherin"

enum_parser = EnumOutputParser(enum=HogwartsHouses)
parser = OutputFixingParser.from_llm(
    llm=llm,
    parser=enum_parser,
    max_retries=3,
    prompt=llama_prompt(
        """
        The next answer was not complient with the instructions provided because of thi error: {error}. 
        Please correct the next answer to follow this instructions: {instructions}, only return one option with no explanation, always reply in lower case.
        {completion}
        """
    )
)

prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.",),
        ("user", 
             """
             Act as the sorting hat and tell to what Hogwarts house will this famous person go: {name}.
             Instructions: {format_instructions} only return one option with no explanation, always reply in Camel case.
             """,)
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = (prompt | llm | parser)

response = chain.invoke({
    "name": "Albert Einstein"
})
response

### Custom Parser

In [ ]:
import emoji
from langchain.schema.output_parser import StrOutputParser

class EmojiOutputParser(StrOutputParser):
    """
    Returns a list of emojis
    """
    def __init__(self):
        super().__init__()

    def parse_result(self, output):
        result = super().parse_result(output)
        return [r["emoji"] for r in emoji.emoji_list(result)]

    def get_format_instructions(self):
        return "First write your response, in a new line translate the response into emojis, use as few emojis as possible and a maximum of 7 emojis. Do not explain your translation"

In [ ]:
parser = EmojiOutputParser()
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.",),
        ("user", "Act as the sorting hat and tell to what Hogwarts house will this famous person go: {name}.\n Always follow this instruction to respond:\n {format_instructions}",),
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"name": "Taylor Swift"})

## 🔗🔗🔗 Encadenando chains:


### Secuencial
Emojis + gussing
```
🔗make emojis
    |
🔗guess emojis
```

In [ ]:
parser = EmojiOutputParser()
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.",),
        ("user", "Act as the sorting hat and tell to what Hogwarts house will this famous person go: {name}.\n Always follow this instruction to respond:\n {format_instructions}",),
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain_emoji = prompt | llm | parser

In [ ]:
parser = EnumOutputParser(enum=HogwartsHouses)

prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.",),
        ("user", """
                Use your knowledge of the Harry Potter to guess which of the four Hogwarts houses are these emojis: {emojis} refering to.
                Instructions: {format_instructions} only return one option with no explanation, always reply in lower case.
            """,)
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain_guess = {"emojis": chain_emoji} | prompt | llm | parser

response = chain_guess.invoke({"name": "Sid Vicious"})
response

### Custom functions
sort + emoji + guessing
```
🔗sort into house
    |
🖥️ python function
    |
🔗guess emojis
```

In [ ]:
from enum import Enum
from langchain.output_parsers.enum import EnumOutputParser

class HogwartsHouses(Enum):
    GRYFFINDOR  = "gryffindor"
    HUFFLEPUFF = "hufflepuff"
    RAVENCLAW = "ravenclaw"
    SLYTHERIN = "slytherin"

enum_parser = EnumOutputParser(enum=HogwartsHouses)
parser = OutputFixingParser.from_llm(
    llm=llm,
    parser=enum_parser,
    max_retries=3,
    prompt=llama_prompt(
        """
        The next answer was not complient with the instructions provided because of thi error: {error}. 
        Please correct the next answer to follow this instructions: {instructions}, only return one option with no explanation, always reply in lower case.
        {completion}
        """
    )
)

prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.",),
        ("user", 
             """
             Act as the sorting hat and tell to what Hogwarts house will this famous person go: {name}.
             Instructions: {format_instructions} only return one option with no explanation, always reply in lower case.
             """,)
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

sorting_chain = prompt | llm | parser

In [ ]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.",),
        ("user", "Use your knowledge of the Harry Potter to guess which of the four Hogwarts houses are these emojis: {emojis} refering to.",)
    ],
)

chain_guess =  prompt | llm | StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableLambda

def house2emoji(house):
    house_map = {
        HogwartsHouses.GRYFFINDOR : '🦁',
        HogwartsHouses.RAVENCLAW : '🐦‍⬛',
        HogwartsHouses.HUFFLEPUFF : '🦡',
        HogwartsHouses.SLYTHERIN  : '🐍'
    }
    return house_map.get(house, "❓")

final_chain = (
    {'name': RunnablePassthrough()} 
    | sorting_chain
    | {"emojis": RunnablePassthrough() | RunnableLambda(house2emoji)}
    | chain_guess
)

Markdown(final_chain.invoke("Rafa Nadal"))

### Flow
Polaridad de una relación
```
      🔗description
        /        \
       /          \
  🔗positive   🔗negative
       \           /
        \         /
         🔗house
```

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

parser = EnumOutputParser(enum=HogwartsHouses)

description = (
    {'name': RunnablePassthrough()}
    | llama_prompt("Describe what is the personality of this famous person: {name}. Only describe the personality. Be brief")
    | llm
    | StrOutputParser()
    | {'description': RunnablePassthrough()}
)

positive = (
    llama_prompt("Given the next personality description of a famous person list 5 positive things of his persona:\n {description}")
    | llm
    | StrOutputParser()
)

negative = (
    llama_prompt("Given the next personality description of a famous person list 5 negative things of his persona:\n {description}")
    | llm
    | StrOutputParser()
)

house = (
    llama_prompt(
        """
        Act as the Sorting Hat and guess to what Hogwarts house will a person go given 5 positives things and 5 negative things his persona.
        positive: {positive}
        negative: {negative}
        Instructions: {format_instructions} only return one option with no explanation, always reply in lower case.
        """,
        system="""
        "You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
        If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
        """,
        format_instructions=parser.get_format_instructions()
    )
    | llm
    | parser
)

chain = (
    description
    | {'positive': positive, 'negative': negative}
    | house
)

response = chain.invoke("Stan Lee")
response

# 📖 Documentos

In [ ]:
sorting_chain = (
    llama_prompt(
        """
        Act as the sorting hat and tell to what Hogwarts house will this person go. Hogwards houses have the next traits:
        Gryffindor: Brave, daring, chivalrous, and bold. They value courage and are often seen as leaders willing to stand up for what’s right.
        Hufflepuff: Loyal, hardworking, fair, and patient. Hufflepuffs prize dedication and kindness, with a strong sense of justice and inclusivity.
        Ravenclaw: Intelligent, curious, creative, and wise. Ravenclaws value learning, wit, and wisdom, always seeking knowledge and understanding.
        Slytherin: Ambitious, cunning, resourceful, and determined. Slytherins are driven, often setting high goals and valuing loyalty within their group.
        Here is the persons biography:
        "{bio}"
        Be brief.
        """,
        system="""
        You are an expert on the Harry Potter universe. Use your bast knowledge of the Harry Potter universe to perform the tasks the user demands. 
        If you don't have the necessary information to perform a task, indicate so, NEVER MADE THINGS UP.
        """
    )
    | llm 
    | StrOutputParser()
)

## 📕 Procesar un documento completo
```
    🔗summarize document
        |
    🔗sorting
```

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

doc = WikipediaLoader(query="Mikel López Iturriaga", lang='es', load_max_docs=1).load()

summarize_chain = llama_prompt("Write a concise summary of the following:\\n\\n{context}") | llm | StrOutputParser()
chain = (
    {"context":RunnablePassthrough()}
    | summarize_chain
    | {"bio": RunnablePassthrough()}
    | sorting_chain
)

Markdown(chain.invoke(doc[0].page_content))

## 🪓 Dividir documentos

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WikipediaLoader

doc = WikipediaLoader(query="Mikel López Iturriaga", lang='es', load_max_docs=1).load()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100
)
chunks = text_splitter.split_documents(doc)

#### Map-Reduce
```
        📕{documento}
       /            \
  📜{cacho_0} .. 📜{cacho_N}
      |                |
🔗{map_chain} .. 🔗{map_chain}
      \              /
     🔗{reduce_chain}
             |
     💬{final_output}
```

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

map_chain = (
    {"context": RunnablePassthrough()}
    | llama_prompt("Write a concise summary of the following:\\n\\n{context}") 
    | llm 
    | StrOutputParser()
)
reduce_chain = (
    {"docs": RunnablePassthrough()}
    | llama_prompt("The following is set of summaries:\n {docs} \nTake these and distill it into a final, consolidated summary. Be brief, 250 words maximun.") 
    | llm 
    | StrOutputParser()
)

map_reduce_chain = (
    RunnablePassthrough()
    | map_chain.map()
    | reduce_chain
)

chain = (
    {"bio": map_reduce_chain}
    | sorting_chain
)

Markdown(chain.invoke(chunks))

#### Refine
```
📕{documento}
 | \
 | 📜{cacho_0}-🔗{initial}-💬{partial_output_0} 
 \                  _________________/
  \                |
   📜{cacho_1}-🔗{refine}-💬{partial_output_1} 
```

In [ ]:
from langchain_core.runnables import RunnableLambda

refine_chain = (
    {"doc": RunnableLambda(lambda x: chunks[0].page_content)}
    | llama_prompt("Summarize the next paragraph. Be brief.\n {doc}")
    | llm
    | StrOutputParser()
)

for ck in chunks[1:]:
    middle_chain = (
        {"summary": RunnablePassthrough(), "paragraph": RunnableLambda(lambda x: ck.page_content)}
        | llama_prompt(
            """"
            Given the next summary with the information processed so far:
            {summary}
            Update it with the next paragraph, be brief:
            {paragraph}
            """
         )
        | llm
        | StrOutputParser()
    )
    refine_chain = refine_chain | middle_chain

chain = (
    {"bio": refine_chain}
    | sorting_chain
)

Markdown(chain.invoke(chunks))

## 📘 Interactuar con un documento (RAG)

### 🪓 Dividir el documento 

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

page_urls = [
    "https://psicologiaymente.com/cultura/dioses-griegos",
    "https://rickriordan.com/extra/meet-the-greek-gods/",
    "https://greektraveltellers.com/blog/the-greek-gods"
]

loader = WebBaseLoader(web_paths=page_urls)
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100
)
chunks = text_splitter.split_documents(docs)
print(len(chunks))

### ️🗄️ Generar la base de datos

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
index = faiss.IndexFlatIP(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
ids = vector_store.add_documents(chunks)
print(len(ids))

### ️⚒️ Extraer cachos relevantes

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

question = "How is Zeus like ?"

docs_chain = RunnablePassthrough() | vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

for doc in docs_chain.invoke(question):
    print("************************************")
    print(doc.page_content)
    print("************************************")

### 🏗️ Procesar documentos
```
    📖 find information
        |
    🔗summarize document
        |
    🔗sorting
```

In [ ]:
from langchain_core.runnables import RunnableLambda

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

question = "How is {0} like ?"
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 6})

chain = (
    {"context": RunnableLambda(lambda x: question.format(x)) | retriever | format_docs, "god": RunnablePassthrough()}
    | llama_prompt(
        """
        Summarize this {god} personality given this context, only use information in the context given: 
        {context}
        """
    )
    | llm
    | StrOutputParser()
    | {"bio": RunnablePassthrough()}
    | sorting_chain
)

Markdown(chain.invoke("Hades"))